## Problem Description

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [2]:
import re
import numpy as np

In [1]:
# Read each row of the input file into a list of strings
with open('input1.txt') as f:
    content = f.readlines()

content = [x.strip() for x in content]
# content

In [4]:
def account_for_edges(idx: int, loc_min: int, loc_max: int):
    return min(max(idx, loc_min), loc_max)

def get_peripherals(start_x: int, end_x: int, loc_y: int, X: list[list]) -> list:
    # Get the peripheral chars of a number with start_x(inclusive) and end_x(non inclusive) location,
    # and loc_y of X matrix to a tuple list
    dirs = [-1, 0, 1]
    ys = [account_for_edges(loc_y+dir, 0, len(X)-1) for dir in dirs]
    xs = list(range(account_for_edges(start_x+dirs[0], 0, len(X[0])), account_for_edges(end_x+dirs[-1], 0, len(X[0]))))
    combos = set([(y, x) for y in ys for x in xs])

    # Account for xs, ys pointing to entry
    number_tuples = set([(loc_y, x) for x in range(start_x, end_x)])
    chars_coor = [(X[y][x], (y,x)) for y,x in list(combos - number_tuples)]
    return chars_coor

In [6]:
def find_gears(X: list[list]):
    gears = []
    for y, line in enumerate(X):
        gear_matches = re.finditer(r'(\*)', line)
        for gear in gear_matches:
            gears.append((gear.group(), (gear.span(), y)))

    return gears

In [2]:
# find_gears(content)

In [8]:
sum = 0
for gear in find_gears(content):
    value = gear[0]
    xs, y = gear[1]
    chars = get_peripherals(*xs, y, content)

    # Check for numbers
    numbers = []
    # Keep track of visited numbers in line by coords of first digit
    idc = []
    for char in chars:

        value = char[0]

        # If we have a digit
        if re.search(r'\d', value) is not None:

            # Get the coordinates of the digit
            y, x = char[1]

            # Look for possible numbers in that line
            possible_matches = re.finditer(r'\d+', content[y])

            # For each of those numbers
            for match in possible_matches:

                # If x is in the range of the span and number not visited
                if x in range(*match.span()) and (y, match.span()[0]) not in idc:
                    numbers.append(int(match.group()))
                    idc.append((y, match.span()[0]))
                    
    if len(numbers) == 2:
        sum += numbers[0]*numbers[1]

sum

86879020